# 包

Go 程序被组织成包。包是**同一目录**中一起编译的源文件的集合。在一个源文件中定义的函数、类型、变量和常量对同一包中的所有其他源文件可见。

Go 的源文件以 `.go` 为后缀名，这些文件名均由小写字母组成，如 `scanner.go`。如果文件名由多个部分组成，则使用下划线 `_` 对它们进行分隔，如 `scanner_test.go` 。文件名不包含空格或其他特殊字符：

    !"#$%&'()*,:;<=>?[\]^`{|}

包中可以导出标识符允许从另一个包访问它。如果**同时**存在以下两种情况，则标识符可导出：

- 标识符名称的第一个字符是 Unicode 大写字母;
- 标识符在包块中声明，或者它是字段名称或方法名称。

## 块

块为匹配的一对大括号括住的声明和语句。

除了源代码中的显式块之外，还有隐式块：

- 全局块 包含所有 Go 源文本。
- 每个包都有一个包块，其中包含该包的所有 Go 源文本。
- 每个文件都有一个文件块，其中包含该文件中的所有 Go 源文本。
- 每个 if 、  for 和 switch 语句都被视为在其自己的隐式块中。
- switch 或 select 语句中的每个子句都充当隐式块。

Go 在词法上使用块限定范围：

- 预先声明的标识符的作用域是全局块。
- 表示在顶层（在任何函数外部）声明的常量、类型、变量或函数（但不是方法）的标识符的作用域是包块。
- 导入的包的包名称的作用域是 包含导入声明的文件的 文件块。
- 表示方法接收者、函数参数或结果变量的标识符的作用域是函数体。
- 在函数内声明的常量或变量标识符的作用域从常量声明或变量声明开始，到包含块的末尾结束。
- 在函数内声明的类型标识符的作用域从类型声明中的标识符开始，到包含块的末尾结束。

在当前块中声明的标识符可以在内部块中重新声明。虽然内部声明的标识符在当前块作用域内，但它已经是内部声明的实体：

In [8]:
import "fmt"

for i := 0; i < 5; i++ { // for 块的 i
    if i := 3; i < 3 { // if 块的 i
        fmt.Print(i) // 所以什么也不打印
    } // if 块结束
    fmt.Println(i) // 打印的是 for 块的 i
}// for 块结束

0
1
2
3
4


## 包组织

### 源文件组织

每个源文件都包含一个包子句，该子句定义它所属的包；后跟一组可能为空的导入声明，声明导入的包；后跟一组可能为空的函数、类型、变量和常量声明：

```
package 包名

import ... 

函数、类型、变量和常量声明 ...
...
...
```

### main 包

main 包是一个特殊的包，不能被导入，它将其他导入包链接起来，创建完整的 Go 程序。一个程序可以只有一个 main 包，`包名` 必须为 `main` 且声明一个无参数无返回值的函数 `main`。

下面是一个 `hello.go` 源文件，只有 main 包，是一个完整的 Go 程序：

In [9]:
// ~/go/src/hello.go
// ~/go 目录是安装 go 时配置的 GOPATH 环境变量
package main

import "fmt" // 导入 fmt 包

func main() {
    fmt.Println("Hello, world.")
}

打开终端，输入命令 `go run ~/go/src/hello.go` 执行程序，将打印出 `Hello, world.`

### 自定义包及导入

在目录 `～/go/src` 下新建文件夹 `mystrings`，在文件夹中新建 go 源文件 `uc.go`，并输入如下内容：

In [11]:
package mystrings

import "strings"

func UpperCase(str string) string {
    return strings.ToUpper(str)
}

文件夹 `mystrings` 中再新建 `reverse.go` 文件，输入如下内容：

In [12]:
package mystrings

func ReverseRunes(s string) string {
    r := []rune(s)
    for i, j := 0, len(r)-1; i < len(r)/2; i, j = i+1, j-1 {
        r[i], r[j] = r[j], r[i]
    }
    return string(r)
}

这样，就自定义了一个包 `mystrings`，包中包含两个源文件 `uc.go` 和 `reverse.go`。

修改 `hello.go` 将自定义包导入，创建一个新的程序：

In [ ]:
package main

import (
    "fmt"
    "mystrings"
)

var str string

func main() {
    fmt.Println("Please enter: ")
    fmt.Scanln(&str)
    input := mystrings.UpperCase(mystrings.ReverseRunes(str))
    fmt.Println(input)
}

打开终端，输入命令 `go run ~/go/src/hello.go` 执行程序，将输出 `Please enter: ` 提示你输入内容。输入 `helloworld` 按回车，所有字母将转为大写并逆序打印出来：`DLROWOLLEH`。

## 构建 Go 程序

打开终端执行命令 `go build ~/go/src/hello.go`，将在 `hello.go` 的同一个文件夹中生成 `hello` 可执行二进制文件。如此 `hello` 就是一个独立可用的小程序啦。

## 包导入方式

包的导入方式有以下三种，以 `fmt` 包为例：

- `import "fmt"` 正常导入

In [1]:
import "fmt"
fmt.Print("ok")

ok

2 <nil>

- `import f "fmt"` 重命名导入，将 fmt 重命名为 f

In [2]:
import f "fmt"
f.Print("ok")

ok

2 <nil>

- `import . "fmt"` 直接导入包内对象（函数、变量等），可直接调用或使用

In [3]:
import . "fmt"
Print("ok")

ok

2 <nil>

使用空白标识符，仅对包进行初始化，而不发生导入行为。例如 `import _ "math"` 仅对 math 包进行初始化。

导入包可以使用包的路径进行导入，甚至可以从远程库中导入并安装包，例如：

    import (
        "example/user/hello/morestrings" // 本地包
        "github.com/google/go-cmp/cmp"   // 远程包
    )